In [26]:
import pandas as pd
import json
import numpy as np
import spacy
from spacy.training.example import Example #holds information for one training instance
import random
from tqdm import tqdm
from spacy.scorer import Scorer
import warnings
warnings.filterwarnings('ignore')

In [27]:
# Authenticate
from google.colab import drive
drive.mount('/content/drive')
from google.colab import auth
auth.authenticate_user()

# Get email of current Colab user
import requests
gcloud_token = !gcloud auth print-access-token
gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
email = gcloud_tokeninfo['email']

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
# Define filepath
if email == 'tamaraexterkate93@gmail.com':
  filename = "/content/drive/MyDrive/TUe/TM/Exports/export_41675_project-41675-at-2023-10-04-09-37-9bbbec63.json"

In [29]:
DATA = []

# read json file
with open(filename, 'rb') as fp:
  training_data = json.load(fp)

  # get text, labels, relations, benchmark for each article
  for article in training_data:
    entities = []
    relations = []
    original_text = article.get('data').get('text')
    if len(article.get('annotations')) == 5: # N=5
      benchmark = True
    else:
      benchmark = False
    for annotation in article.get('annotations'):
      if annotation.get('ground_truth') == True: # only include ground truth
        for ind, label in enumerate(annotation.get('result')):
          if label.get('type') == 'labels':
            start = label.get('value').get('start')
            end = label.get('value').get('end')
            label = label.get('value').get('labels')[0] # note: cannot deal with multiple labels
            entities.append((start, end, label))
          elif label.get('type') == 'relation':
            from_id = label.get('from_id')
            to_id = label.get('to_id')
            relation = label.get('labels')[0] # note: cannot deal with multiple relations
            relations.append((from_id, to_id, relation))
          else:
            print("found unknown label type (no label or relation)")

    # append article to training data as dictionary
    DATA.append({
    'text': original_text,
    'entities': entities,
    'relations': relations,
    'benchmark': benchmark
    })

In [30]:
# TODO: deal with overlapping entities
DATA[1]

{'text': 'Max Karl Ernst Ludwig Planck (23 April 1858 – 4 October 1947) was a German theoretical physicist whose discovery of energy quanta won him the Nobel Prize in Physics in 1918.\nPlanck made many substantial contributions to theoretical physics, but his fame as a physicist rests primarily on his role as the originator of quantum theory, which revolutionized human understanding of atomic and subatomic processes. In 1948, the German scientific institution Kaiser Wilhelm Society (of which Planck was twice president) was renamed Max Planck Society (MPG). The MPG now includes 83 institutions representing a wide range of scientific directions.\nLife and career.\nPlanck came from a traditional, intellectual family. His paternal great-grandfather and grandfather were both theology professors in Göttingen; his father was a law professor at the University of Kiel and Munich. One of his uncles was also a judge.',
 'entities': [(0, 28, 'Winner'),
  (30, 43, 'Date'),
  (46, 60, 'Date'),
  (68

In [31]:
model = None
if model is not None:
  nlp = spacy.load(model)
  print(f"Loaded model '{model}'")
else:
  nlp = spacy.blank('en')
  print("Created blank 'en' model")

# set up the pipeline
if 'ner' not in nlp.pipe_names:
  ner = nlp.add_pipe('ner')
# nlp.add_pipe(ner, last=True)
else:
  ner = nlp.get_pipe('ner')

Created blank 'en' model
